In [ ]:
# mount drive https://datascience.stackexchange.com/questions/29480/uploading-images-folder-from-my-system-into-google-colab
# login with your google account and type authorization code to mount on your google drive.
import os
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
# Specify the directory path where `assignemnt1.ipynb` exists.
# For example, if you saved `assignment1.ipynb` in `/gdrive/My Drive/cs470/assignment1` directory,
# then set root = '/gdrive/My Drive/cs470/assignment1'
root = '/gdrive/My Drive/colab'

In [ ]:
from PIL import Image
from tqdm import tqdm
from pathlib import Path
import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import requests
from torchvision.datasets import CIFAR10
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier


일단 모델에 넣을 값들을 정하는게 좋을 듯.
예를 들자면 뭐 상대편 타워 깨진 개수와 우리팀 타워깨진 개수, 상대팀과 우리팀의 평균 레벨 차이, 돈 차이, 퍼블은 어느쪽이 먹었는지 등... 그런걸 좀 정리해서 모델에 넣는다고 생각하고 모델을 설계하고 데이터를 정리하는게 필요할 듯

In [ ]:
api_key = 'RGAPI-9fce769c-49e1-4cc2-8574-086024c85ae4'

master = 'https://kr.api.riotgames.com/lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5?api_key=' + api_key
r0 = requests.get(master)
league_df0 = pd.DataFrame(r0.json())
league_df0.reset_index(inplace=True)
league_entries_df0 = pd.DataFrame(dict(league_df0['entries'])).T 
league_df0 = pd.concat([league_df0, league_entries_df0], axis=1) 
league_df0 = league_df0.drop(['index', 'queue', 'name', 'leagueId', 'entries', 'rank'], axis=1)


grandmaster = 'https://kr.api.riotgames.com/lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5?api_key=' + api_key
r1 = requests.get(grandmaster)
league_df1 = pd.DataFrame(r1.json())
league_df1.reset_index(inplace=True)
league_entries_df1 = pd.DataFrame(dict(league_df1['entries'])).T 
league_df1 = pd.concat([league_df1, league_entries_df1], axis=1) 
league_df1 = league_df1.drop(['index', 'queue', 'name', 'leagueId', 'entries', 'rank'], axis=1)

challenger = 'https://kr.api.riotgames.com/lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5?api_key=' + api_key
r2 = requests.get(challenger)
league_df2 = pd.DataFrame(r2.json())
league_df2.reset_index(inplace=True)
league_entries_df2 = pd.DataFrame(dict(league_df2['entries'])).T 
league_df2 = pd.concat([league_df2, league_entries_df2], axis=1)
league_df2 = league_df2.drop(['index', 'queue', 'name', 'leagueId', 'entries', 'rank'], axis=1)



league_df = pd.concat([league_df0,league_df1])
league_df = pd.concat([league_df,league_df2])
league_df.to_csv(root + "/league_df.csv")

league_df['account_id'] = 0
#accountid
for i in range(50):
    try:
        sohwan = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/' + league_df['summonerName'].iloc[i] + '?api_key=' + api_key 
        r = requests.get(sohwan)
        
        while r.status_code == 429:
            time.sleep(5)
            sohwan = 'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/' + league_df['summonerName'].iloc[i] + '?api_key=' + api_key 
            r = requests.get(sohwan)
            
        account_id = r.json()['accountId']
        league_df.iloc[i,-1] = account_id
    
    except:
        pass

#gameid
match_info_df = pd.DataFrame()
season = str(13)
for i in range(length(league_df)):
    try:
        match0 = 'https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/' + league_df['account_id'].iloc[i]  +'?season=' + season + '&api_key=' + api_key
        r = requests.get(match0)
        while r.status_code == 429:
            time.sleep(5)
            match0 = 'https://kr.api.riotgames.com/lol/match/v4/matchlists/by-account/' + league_df['account_id'].iloc[i]  +'?season=' + season + '&api_key=' + api_key
            r = requests.get(match0)
            
        match_info_df = pd.concat([match_info_df, pd.DataFrame(r.json()['matches'])])
        print(match_info_df)
    except:
        print(i)
print(match_info_df)

In [ ]:
match_fin = pd.DataFrame()
# for i in range(len(match_info_df)):
for i in range(5000):  

    api_url='https://kr.api.riotgames.com/lol/match/v4/matches/' + str(match_info_df['gameId'].iloc[i]) + '?api_key=' + api_key
    r = requests.get(api_url)

    if r.status_code == 200: 
        pass

    elif r.status_code == 429:
        print('api cost full : infinite loop start')
        print('loop location : ',i)
        start_time = time.time()

        while True: 
            if r.status_code == 429:

                print('try 10 second wait time')
                time.sleep(10)

                r = requests.get(api_url)
                print(r.status_code)

            elif r.status_code == 200: 
                print('total wait time : ', time.time() - start_time)
                print('recovery api cost')
                break

    elif r.status_code == 503: 
        print('service available error')
        start_time = time.time()

        while True:
            if r.status_code == 503 or r.status_code == 429:

                print('try 10 second wait time')
                time.sleep(10)

                r = requests.get(api_url)
                print(r.status_code)

            elif r.status_code == 200: 
                print('total error wait time : ', time.time() - start_time)
                print('recovery api cost')
                break
    elif r.status_code == 403: 
        print('you need api renewal')
        print('break')
        break

    mat = pd.DataFrame(list(r.json().values()), index=list(r.json().keys())).T
    match_fin = pd.concat([match_fin,mat])
print(match_fin)

In [ ]:
def riot_timeline_dataset(pre_dataset,api_key,timeline):
    
    challenger_game = pre_dataset
    my_api = api_key
    
    use_columns = ['gameId','blueWins','blueTotalLevel','blueKill','bluedeath','blueTowerKills','blueDragon','redWins','redTotalLevel','redTowerKills','redDragon']

    challenger_timeline_df1 = pd.DataFrame()
    challenger_timeline_df = pd.DataFrame()
    error_list = []
    match_url = 'https://kr.api.riotgames.com/lol/match/v4/timelines/by-match/{}?api_key=' +  my_api

    for b in range(5000):

        try:
            game_id = challenger_game['gameId'].iloc[b]
            req = requests.get(match_url.format(game_id))
            print('status : ' + str(req.status_code) + '  loop location : ' + str(b))
            
            frames = req.json()['frames']
            
            #시작하고 n분
            lc0 = 0 # 위치
            while True:
                try:
                    timestamps = frames[lc0]['timestamp']
                    if timestamps <= timeline: # n Minute
                        lc0 += 1
                    else:
                        lc = lc0-1
                        break
                except:
                    lc = lc0 - 1
                    break

            participant = frames[lc]['participantFrames']

            bluetotal_level = []
            redtotal_level = []

            for i in range(len(participant)):
                i = i+1
                if 1 <=participant[str(i)]['participantId'] <= 5:
                    bluetotal_level.append(participant[str(i)]['level'])
                else:
                    redtotal_level.append(participant[str(i)]['level'])
            
            
            blue_kill, blue_death = 0,0
            blue_dragon, red_dragon = 0,0
            blue_tower,red_tower = 0,0
            bluewin, redwin = 0,0

            for y in range(1,lc+1):
                events = frames[y]['events']

                for x in range(len(events)):
                    if events[x]['type'] == 'CHAMPION_KILL': 
                        if 1 <= events[x]['killerId'] <= 5:
                            blue_kill += 1
                        else:
                            blue_death += 1

                    elif events[x]['type'] == 'ELITE_MONSTER_KILL':
                        if 1 <= events[x]['killerId'] <= 5:
                            if events[x]['monsterType']== 'DRAGON':
                                blue_dragon += 1
                        else:
                            if events[x]['monsterType']== 'DRAGON':
                                red_dragon += 1

                    elif events[x]['type'] == 'BUILDING_KILL':
                        if 1 <= events[x]['killerId'] <= 5:
                            if events[x]['buildingType'] == 'TOWER_BUILDING':
                                blue_tower += 1
                        else:
                            if events[x]['buildingType'] == 'TOWER_BUILDING':
                                red_tower += 1
            
            if (challenger_game['teams'].iloc[b])[0]['win'] == 'Fail': #bluelose
              bluewin = 0
              redwin = 1
            elif (challenger_game['teams'].iloc[b])[0]['win'] == 'Win':  #bluewin
              bluewin = 1
              redwin = 0

            data_list = [game_id,bluewin,np.sum(bluetotal_level),blue_kill,blue_death,blue_tower,blue_dragon,redwin,np.sum(redtotal_level),red_tower,red_dragon]
            challenger_timeline_df0 = pd.DataFrame(np.array([data_list]), columns = use_columns)   
            challenger_timeline_df1 = challenger_timeline_df1.append(challenger_timeline_df0)

            print('data crawling success')

            if b != 0 and b % 2000 == 0 :
                challenger_timeline_df = challenger_timeline_df.append(challenger_timeline_df1)
                challenger_timeline_df1 = pd.DataFrame()

        except: #에러발생 시 걍 넘어감
            print('error visual')
            error_list.append(b)
            pass
    challenger_timeline_df = challenger_timeline_df.append(challenger_timeline_df1)
    
    return challenger_timeline_df

In [ ]:
timeline_df = riot_timeline_dataset(match_fin,api_key,900000)
timeline_df.to_csv(root + "/timeline_df.csv")

In [ ]:
#https://kr.api.riotgames.com/lol/match/v4/matches/4247263043?api_key=RGAPI-329d019f-4e50-4fe1-93d6-7a14a66d4583
#YOU SHOULD NOT RUN THIS CELL

df = pd.read_csv(root + '/timeline_df.csv')

my_df = pd.DataFrame()
my_df['gameId'] = df['gameId']
my_df['win'] = df['blueWins']
my_df['diffLevel'] = df['blueTotalLevel'] - df['redTotalLevel']
my_df['diffKill'] = df['blueKill'] - df['blueDeath']
my_df['diffTower'] = df['blueTowerKills'] - df['redTowerKills']
my_df['diffDragon'] = df['blueDragon'] - df['redDragon']
print(my_df)

winrate_list = {266 : ("Aatrox", 46.07), 103 : ("Ahri", 50.59), 84 : ("Akali", 48.85),12 : ("Alistar", 50.27), 32 : ("Amumu", 53.24), 34 : ("Anivia", 54.33), 1 : ("Annie", 47.58), 523 : ("Aphelios", 53.4), 22 : ("Ashe", 49.57), 136 : ("AurelionSol", 44.32), 268 : ("Azir", 54.58), 432 : ("Bard", 54.09), 53 : ("Blitzcrank", 46.99), 63 : ("Brand", 50.24), 201 : ("Braum", 56.19), 51 : ("Caitlyn", 48.48), 164 : ("Camile", 50.58), 69 : ("Cassiopeia", 52.37), 31 : ("Chogath", 50.17), 42 : ("Corki", 50.89), 122 : ("Darius", 54.97), 131 : ("Diana", 49.24), 119 : ("Draven", 51.61), 36 : ("DrMundo", 51.46), 245 : ("Ekko", 50.67), 60 : ("Elise", 49.19), 28 : ("Evelynn", 49.54), 81 : ("Ezreal", 47.24), 9 : ("Fiddlesticks", 52.9), 114 : ("Fiora", 50.27), 105 : ("Fizz", 51.33), 3 : ("Galio", 47.29), 41 : ("Gangplank", 45.86), 86 : ("Garen", 47.19), 150 : ("Gnar", 55.04), 79 : ("Gragas", 43.72), 104 : ("Graves", 49.45), 120 : ("Hecarim", 47.45), 74 : ("Heimerdinger", 51.27), 420 : ("Illaoi", 52.8), 39 : ("Irelia", 42.59), 427 : ("Ivern", 44.7), 40 : ("Janna", 47.79), 59 : ("JarvanIV", 46.99), 24 : ("Jax", 45.92), 126 : ("Jayce", 52.2), 202 : ("Jhin", 51.11), 222 : ("Jinx", 48.95), 145 : ("Kaisa", 50), 429 : ("Kalista", 46.35), 43 : ("Karma", 47.47), 30 : ("Karthus", 48.71), 38 : ("Kassadin", 54.94), 55 : ("Katarina", 49.53), 10 : ("Kayle", 57.83), 141 : ("Kayn", 52.49), 85 : ("Kennen", 52.4), 121 : ("Khazix", 50.16), 203 : ("Kindred", 51.16), 240 : ("Kled", 51.58), 96 : ("KogMaw", 42.54), 7 : ("Leblanc", 49.97), 64 : ("LeeSin", 47.77), 89 : ("Leona", 50.75), 876 : ("Lillia", 47.75), 127 : ("Lissandra", 42.01), 236 : ("Lucian", 48.19), 117 : ("Lulu", 51.81), 99 : ("Lux", 48.26), 54 : ("Malphite", 50.72), 90 : ("Malzahar", 53.56), 57 : ("Maokai", 53.79), 11 : ("MasterYi", 48.12), 21 : ("MissFortune", 53.23), 62 : ("MonkeyKing", 49.26), 82 : ("Mordekaiser", 51.24), 25 : ("Morgana", 48.29), 267 : ("Nami", 48.42), 75 : ("Nasus", 48.66), 111 : ("Nautilus", 47.63), 518 : ("Neeko", 50.85), 76 : ("Nidalee", 48.05), 56 : ("Nocturne", 50.76), 20 : ("Nunu", 51.48), 2 : ("Olaf", 47.78), 61 : ("Orianna", 48.73), 516 : ("Ornn", 55.78), 80 : ("Pantheon", 47.5), 78 : ("Poppy", 47.64), 555 : ("Pyke", 46.6), 246 : ("Qiyana", 48.38), 133 : ("Quinn", 49.94), 497 : ("Rakan", 46.97), 33 : ("Rammus", 54.14), 421 : ("RekSai", 46.06), 58 : ("Renekton", 45.85), 107 : ("Rengar", 49.17), 92 : ("Riven", 51.62), 68 : ("Rumble", 50.2), 13 : ("Ryze", 47.67), 360 : ("Samira", 48.92), 113 : ("Sejuani", 50.69), 235 : ("Senna", 48.49), 147 : ("Seraphine", 51.25), 875 : ("Sett", 47.67), 35 : ("Shaco", 50.35), 98 : ("Shen", 51.07), 102 : ("Shyvana", 50.75), 27 : ("Singed", 51.35), 14 : ("Sion", 48.5), 15 : ("Sivir", 45.58), 72 : ("Skarner", 53.79), 37 : ("Sona", 56.07), 16 : ("Soraka", 52.05), 50 : ("Swain", 50.06), 517 : ("Sylas", 49.95), 134 : ("Syndra", 50.26), 223 : ("TahmKench", 58.37), 163 : ("Taliyah", 60.31), 91 : ("Talon", 49.37), 44 : ("Taric", 51.54), 17 : ("Teemo", 55.34), 412 : ("Thresh", 50.32), 18 : ("Tristana", 49.44), 48 : ("Trundle", 48.66), 23 : ("Tryndamere", 49.23), 4 : ("TwistedFate", 51.53), 29 : ("Twitch", 47.51), 77 : ("Udyr", 51.48), 6 : ("Urgot", 48.81), 110 : ("Varus", 46.22), 67 : ("Vayne", 54.34), 45 : ("Veigar", 55.74), 161 : ("Velkoz", 46.98), 254 : ("Vi", 54.12), 112 : ("Viktor", 51.93), 8 : ("Vladimir", 47.38), 106 : ("Volibear", 50.92), 19 : ("Warwick", 48.26), 498 : ("Xayah", 50.55), 101 : ("Xerath", 50.63), 5 : ("XinZhao", 49.71), 157 : ("Yasuo", 47.79), 777 : ("Yone", 49.74), 83 : ("Yorick", 47.69), 350 : ("Yuumi", 54.96), 154 : ("Zac", 49.89), 238 : ("Zed", 50.77), 115 : ("Ziggs", 39.52), 26 : ("Zilean", 51.02), 142 : ("Zoe", 48.62), 143 : ("Zyra", 48.91)}
winrate_diff_list = []

for i in range(5000):
  matchid = str(my_df['gameId'][i])
  api_url='https://kr.api.riotgames.com/lol/match/v4/matches/' + matchid +'?api_key=' + api_key
  r = requests.get(api_url)

  blue_winrate_total = 0.0
  red_winrate_total = 0.0
  for j in range(10):
    try :
      teamid = r.json()['participants'][j]['teamId']
    except KeyError :
      print("Key Erorr happened")
      blue_winrate_total = 0
      red_winrate_total = 0
      break
    if teamid == 100: #blue team
      blue_winrate_total += winrate_list[r.json()['participants'][j]['championId']][1]
    elif teamid == 200: #red team
      red_winrate_total += winrate_list[r.json()['participants'][j]['championId']][1]
  winrate_diff = blue_winrate_total - red_winrate_total
  winrate_diff_list.append(winrate_diff)
  if (i % 50 == 0) :
      print ("Calculating " + str(i) + " of 10000")
  time.sleep(0.9)

my_df['diffWinrate'] = winrate_diff_list
print(my_df)
my_df.to_csv(root + "/new_dataframe.csv")
print("save end")